#  TRAINING MODEL

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#for modelling
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [3]:
df = pd.read_csv('data/raw.csv')

In [4]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group D,some college,standard,completed,59,70,78
1,male,group D,associate's degree,standard,none,96,93,87
2,female,group D,some college,free/reduced,none,57,76,77
3,male,group B,some college,free/reduced,none,70,70,63
4,female,group D,associate's degree,standard,none,83,85,86


In [7]:
X = df.drop(columns=['math score'], axis=1)

In [8]:
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group D,some college,standard,completed,70,78
1,male,group D,associate's degree,standard,none,93,87
2,female,group D,some college,free/reduced,none,76,77
3,male,group B,some college,free/reduced,none,70,63
4,female,group D,associate's degree,standard,none,85,86


In [9]:
y = df['math score']

In [10]:
y

0      59
1      96
2      57
3      70
4      83
       ..
995    77
996    80
997    67
998    80
999    58
Name: math score, Length: 1000, dtype: int64

In [11]:
# Create Column transformer with 3 types of transformers
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

num_transformer = StandardScaler()
cat_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", cat_transformer, cat_features),
        ("StandardScalar", num_transformer, num_features),
    ]
)

In [12]:
X = preprocessor.fit_transform(X)

In [13]:
X.shape

(1000, 19)

In [14]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  0.        ,
        -0.02709151,  0.58994292],
       [ 0.        ,  1.        ,  0.        , ...,  1.        ,
         1.60407283,  1.18920774],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.39842962,  0.52335794],
       ...,
       [ 1.        ,  0.        ,  1.        , ...,  0.        ,
         1.10763151,  1.12262276],
       [ 0.        ,  1.        ,  0.        , ...,  1.        ,
         0.11474887, -0.47541676],
       [ 0.        ,  1.        ,  0.        , ...,  1.        ,
        -1.65825585, -1.60736142]])

In [15]:
# Separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

## Create an Evaluate Function to give all metrics after Model Training

In [16]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_scor = r2_score(true, predicted)
    return mae, rmse, r2_scor

In [18]:
models = {
    "LinearRegression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge":Ridge(),
    "K- Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(),
    "AdaBoostRegressor": AdaBoostRegressor()
}
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train) # Train Model

    # make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate Train and Test dataset
    model_train_mae, model_train_rsme, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rsme, model_test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Performance for Training set')
    print(" - Root Mean Squared Error : {:.4}".format(model_train_rsme))
    print(" - Mean Absolute Error : {:.4}".format(model_train_mae))
    print(" - R2 Score : {:.4}".format(model_train_r2))

    print("--------------------------------------------------------------------------------------------")

    print('Model Performance for Testing set')
    print(" - Root Mean Squared Error : {:.4}".format(model_test_rsme))
    print(" - Mean Absolute Error : {:.4}".format(model_test_mae))
    print(" - R2 Score : {:.4}".format(model_test_r2))
    r2_list.append(model_test_r2)

    print('='*35)
    print('\n')

LinearRegression
Model Performance for Training set
 - Root Mean Squared Error : 5.472
 - Mean Absolute Error : 4.389
 - R2 Score : 0.8723
--------------------------------------------------------------------------------------------
Model Performance for Testing set
 - Root Mean Squared Error : 4.984
 - Mean Absolute Error : 4.074
 - R2 Score : 0.8875


Lasso
Model Performance for Training set
 - Root Mean Squared Error : 6.721
 - Mean Absolute Error : 5.458
 - R2 Score : 0.8074
--------------------------------------------------------------------------------------------
Model Performance for Testing set
 - Root Mean Squared Error : 6.809
 - Mean Absolute Error : 5.376
 - R2 Score : 0.79


Ridge
Model Performance for Training set
 - Root Mean Squared Error : 5.472
 - Mean Absolute Error : 4.388
 - R2 Score : 0.8723
--------------------------------------------------------------------------------------------
Model Performance for Testing set
 - Root Mean Squared Error : 4.987
 - Mean Absol

In [23]:
pd.DataFrame(list(zip(model_list,r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"])

,Model Name,R2_Score
4,Decision Tree,0.744206
1,Lasso,0.789992
3,K- Neighbors Regressor,0.810209
6,XGBRegressor,0.826102
8,AdaBoostRegressor,0.833613
5,Random Forest Regressor,0.855220
7,CatBoosting Regressor,0.874470
2,Ridge,0.887328
0,LinearRegression,0.887488


### Differences between Actual and Predicted Values

In [24]:
pred_df = pd.DataFrame({'Actual Value':y_test, 'Predicted Value':y_test_pred, 'Difference':y_test_pred-y_test})
pred_df

,Actual Value,Predicted Value,Difference
521,67,73.052632,6.052632
737,78,82.276119,4.276119
740,39,45.173913,6.173913
660,70,72.016000,2.016000
411,90,82.276119,-7.723881
...,...,...,...
408,100,88.041667,-11.958333
332,66,70.815068,4.815068
208,66,63.757669,-2.242331
613,68,71.781250,3.781250
